Copyright (c) 2020-2021. All rights reserved.

Licensed under the MIT License.

# HPO for Fine-Tuning Pre-trained Language Models


## 1. Introduction


In this notebook, we demonstrate a procedure for troubleshooting HPO failure in fine-tuning pre-trained language models (introduced in the following paper):

*An Empirical Study on Hyperparameter Optimization for Fine-Tuning Pre-trained Language Models. Xueqing Liu, Chi Wang. To appear in ACL-IJCNLP 2021*

FLAML requires `Python>=3.6`. To run this notebook example, please install flaml with the `notebook` and `nlp` options:
```bash
pip install flaml[notebook,nlp]
```

In [4]:
!pip install flaml[notebook,nlp]


/Users/xliu/Documents/xqq/Documents/work/projects/automl/FLAML/notebook


## 2. Initial Experimental Study (Section 4)


### Load dataset 

Load the dataset using AutoTransformer.prepare_data. In this notebook, we use the Recognizing Textual Entailment (RTE) dataset and the Electra model as an example:

In [9]:
from flaml.nlp.autotransformers import AutoTransformers
autohf = AutoTransformers()
preparedata_setting = {
        "dataset_subdataset_name": "glue:rte",
        "pretrained_model_size": "google/electra-base-discriminator:base",
        "data_root_path": "data/",
        "max_seq_length": 128,
        }
autohf.prepare_data(**preparedata_setting)


Reusing dataset glue (/home/xliu127/.cache/huggingface/datasets/glue/rte/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/rte/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-404440ecda81d9a6.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/rte/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-778149280c71c970.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/rte/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-4b5989072a90417b.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/rte/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-404440ecda81d9a6.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/rte/1.0.0/7c99657241149a

First, we run grid search using Electra. By specifying `algo_mode="grid"`, AutoTransformers will by default use the grid configuration recommended for the Electra model:

In [11]:
autohf_settings = {"resources_per_trial": {"gpu": 1, "cpu": 1},
                    "num_samples": -1, # unlimited sample size
                    "time_budget": 3600,
                    "ckpt_per_epoch": 1,
                    "fp16": True,
                    "algo_mode": "grid"
                   }
validation_metric, analysis = autohf.fit(**autohf_settings,)

The value for metric_name is not specified, setting it to the default value accuracy. Alternatively, you can set it to accuracy,loss
The value for metric_mode_name is not specified, setting it to the default value max. Alternatively, you can set it to max,min
2021-06-13 11:43:59,466	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 3.0.0 (/data/installation/anaconda3/envs/test/lib/python3.7/site-packages), Requirement.parse('pyarrow<2.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).


AssertionError: 

Run HPO for 1GST:

In [ ]:
autohf_settings = {"resources_per_trial": {"gpu": 1, "cpu": 1},
                    "num_samples": -1, # unlimited sample size
                    "time_budget": time_budget,
                    "ckpt_per_epoch": 5,
                    "fp16": True,
                   }
validation_metric, analysis = autohf.fit(**autohf_settings,)
print(validation_metric)

: